In [358]:
"""
This notebook works through the logic for pre & post cooling
"""

'\nThis notebook works through the logic for pre & post cooling\n'

In [359]:
import collections

import numpy as np
import pandas as pd

In [360]:
class env(object):
    def __init__(self):
        self.memory = 3
        self.capacity = 5
        self.relaxation_time = 2
        
        self.info, self.outputs = {}, {}
        
        self.reset()

    def reset(self):    
        self.precool_hist = collections.deque([0], maxlen=self.memory)
        self.postcool_hist = collections.deque([0], maxlen=self.memory)
        self.relaxation_hist = collections.deque([0], maxlen=self.relaxation_time)
        
        self.info['demand_adjustment_hist'] = []
        self.info['precool_hist'] = []
        self.info['postcool_hist'] = []
        self.info['relaxation_hist'] = []
        
        print('reset')
        return None

    def update_hists(self, precool, postcool, relaxation):
            self.precool_hist.append(precool)
            self.postcool_hist.append(postcool)
            self.relaxation_hist.append(relaxation)
            return None    

    def update_info(self, demand_adjustment): 
        """
        updates the self.info dictionary
        """
        self.info['demand_adjustment_hist'].append(demand_adjustment)
        self.info['precool_hist'].append(self.precool_hist[-1])
        self.info['postcool_hist'].append(self.postcool_hist[-1])
        self.info['relaxation_hist'].append(self.relaxation_hist[-1])
        return self.info
    
    def output_info(self):
        """
        extracts info and turns into dataframes & graphs
        """
        def demand_adjustment_fig():
            return fig
        
        index = np.arange(0, len(self.info['demand_adjustment_hist']))
        
        total_demand_adjustment = sum(self.info['demand_adjustment_hist'])
        print('total demand adjustment was {}'.format(total_demand_adjustment))
        
        self.outputs['dataframe'] = pd.DataFrame(data=[], index=index, columns=)
        
        self.outputs['dataframe'] = pd.DataFrame.from_dict(self.info)

        
#         self.outputs['demand_adjustment_hist'] = pd.DataFrame(self.info['demand_adjustment_hist'], index)
#         self.outputs['precool_hist'] = pd.DataFrame(self.info['precool_hist'], index)
#         self.outputs['postcool_hist'] = pd.DataFrame(self.info['postcool_hist'], index)
#         self.outputs['relaxation_hist'] = pd.DataFrame(self.info['relaxation_hist'], index)
        
        # self.outputs['demand_adjustment_fig'] = demand_adjustment_fig()
        
        return self.outputs
        
    def step(self,action):
        
        """
        this function steps through the environment
        
        TODO - a way to delay the start of the next precooling after the postcooling ended
        """
    
        precool_hist = sum(self.precool_hist)
        postcool_hist = sum(self.postcool_hist)
        relaxation_hist = sum(self.relaxation_hist)
        
        cooling_demand = 0

        #  should we start a precooling event
        if precool_hist == 0 and action == 1 and self.postcool_hist[-1] == 0 and relaxation_hist == 0:
            print('starting pre-cooling event')
            self.update_hists(1, 0, 0)

        #  are we already in a precooling event
        elif precool_hist > 0 and precool_hist < self.memory and self.precool_hist[-1] == 1:
            print('continuing pre-cooling event')
            self.update_hists(1, 0, 0)

        #  are we at the end of a precooling event
        #  aka should we start a postcooling event
        elif precool_hist == self.memory:
            print('ending pre-cooling event')
            print('starting post-cooling event')
            self.update_hists(0, 1, 0)

        #  are we already in a postcooling event   
        elif postcool_hist > 0 and postcool_hist < self.memory and self.postcool_hist[-1] == 1:    
            print('continuing post-cooling event')
            self.update_hists(0, 1, 0)

        #  are we ending a postcooling event
        elif postcool_hist == self.memory and precool_hist == 0:
            print('ending post-cooling event')
            self.update_hists(0, 0, 1)

        else:
            print('nothing is happening')
            self.update_hists(0, 0, 0)
            
            if relaxation_hist > 0:
                print('in relaxation time')
            
        print('pre-cooling history {}'.format(self.precool_hist))
        print('post-cooling history {}'.format(self.postcool_hist)) 
        print('relaxation history {}'.format(self.relaxation_hist)) 
        
        #  now calculating the effect on demand
        demand_adjustment = 0
        
        if self.precool_hist[-1] == 1:
            demand_adjustment = self.capacity
            
        if self.postcool_hist[-1] == 1:
            demand_adjustment = -self.capacity    
            
        print('demand adjustment is {}'.format(demand_adjustment))    
        
        #  saving info
        self.info = self.update_info(demand_adjustment)
        
        #  check cooling balance
            
        return None    

In [361]:
e = env()

reset


In [362]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0], maxlen=3)
post-cooling history deque([0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [363]:
e.step(1)

starting pre-cooling event
pre-cooling history deque([0, 0, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [364]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([0, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [365]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([1, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [366]:
e.step(1)

ending pre-cooling event
starting post-cooling event
pre-cooling history deque([1, 1, 0], maxlen=3)
post-cooling history deque([0, 0, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [367]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([1, 0, 0], maxlen=3)
post-cooling history deque([0, 1, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [368]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [369]:
e.step(1)

ending post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 0], maxlen=3)
relaxation history deque([0, 1], maxlen=2)
demand adjustment is 0


In [370]:
e.step(1)

nothing is happening
in relaxation time
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 0, 0], maxlen=3)
relaxation history deque([1, 0], maxlen=2)
demand adjustment is 0


In [371]:
e.step(1)

nothing is happening
in relaxation time
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [372]:
e.step(1)

starting pre-cooling event
pre-cooling history deque([0, 0, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [373]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([0, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [374]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([1, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [375]:
e.step(1)

ending pre-cooling event
starting post-cooling event
pre-cooling history deque([1, 1, 0], maxlen=3)
post-cooling history deque([0, 0, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [376]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([1, 0, 0], maxlen=3)
post-cooling history deque([0, 1, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [377]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [378]:
e.step(1)

ending post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 0], maxlen=3)
relaxation history deque([0, 1], maxlen=2)
demand adjustment is 0


In [379]:
e.step(1)

nothing is happening
in relaxation time
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 0, 0], maxlen=3)
relaxation history deque([1, 0], maxlen=2)
demand adjustment is 0


In [380]:
e.step(1)

nothing is happening
in relaxation time
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [381]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [382]:
e.step(1)

starting pre-cooling event
pre-cooling history deque([0, 0, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [383]:
e.step(0)

continuing pre-cooling event
pre-cooling history deque([0, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [384]:
e.step(0)

continuing pre-cooling event
pre-cooling history deque([1, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 5


In [385]:
e.step(0)

ending pre-cooling event
starting post-cooling event
pre-cooling history deque([1, 1, 0], maxlen=3)
post-cooling history deque([0, 0, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [386]:
e.step(0)

continuing post-cooling event
pre-cooling history deque([1, 0, 0], maxlen=3)
post-cooling history deque([0, 1, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [387]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 1], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is -5


In [388]:
e.step(0)

ending post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 0], maxlen=3)
relaxation history deque([0, 1], maxlen=2)
demand adjustment is 0


In [389]:
e.step(0)

nothing is happening
in relaxation time
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 0, 0], maxlen=3)
relaxation history deque([1, 0], maxlen=2)
demand adjustment is 0


In [390]:
e.step(0)

nothing is happening
in relaxation time
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [391]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [392]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [393]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [394]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
relaxation history deque([0, 0], maxlen=2)
demand adjustment is 0


In [395]:
outputs = e.output_info()
demand_adj = outputs['demand_adjustment_hist']

total demand adjustment was 0


In [396]:
demand_adj

,0
0,0
1,5
2,5
3,5
4,-5
5,-5
6,-5
7,0
8,0
9,0
